In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import unicodedata

## GET STARTING PAGE WITH UNIVERSITIES

In [2]:
r = requests.get('https://www.felvi.hu/felveteli/egyetemek_foiskolak/!IntezmenyiOldalak/intezmeny_lista.php?elj=20a')

In [3]:
soup = BeautifulSoup(r.content, 'html5lib')

In [4]:
# get subpage url of university programs
def extract_uni_and_href(soup, base_url='https://www.felvi.hu'):
    out = {}
    trs = soup.findAll('tr')
    
    for tr in trs:
        try:
            tr.findAll('td')[0]['title']
            for i, td in enumerate(tr.findAll('td')):
                if i in (0,2):
                    continue
                elif i in (1,3):
                    title = td['title']
                    href = td.findAll('a')[0]['href']
                    out[title] = f"{base_url}{href}".replace('&elj=20a', '&oldal=2&elj=20a')
        except KeyError:
            continue
        
    return out

In [12]:
uni_links = extract_uni_and_href(soup)

## GET SUBPAGES IF DEPARTMENTS HAVE DIFFERENT SUBPAGES

In [26]:
def try_extract_subpages(uni_links, base_url='https://www.felvi.hu'):
    new_uni_links = {}
    keys_to_delete = []
    for k, v in uni_links.items():
        r = requests.get(v)
        soup = BeautifulSoup(r.content, 'html5lib')
        subpages = [elem for elem in soup.findAll('a') if k in elem.text]    
        if len(subpages) > 0:
            keys_to_delete.append(k)
            for elem in subpages:
                href = elem['href']
                new_uni_links[unicodedata.normalize("NFKD", elem.text).strip()] = f"{base_url}{href}".replace('&elj=20a', '&oldal=2&elj=20a')
    return new_uni_links, keys_to_delete

In [27]:
new_uni_links, keys_to_delete = try_extract_subpages(uni_links)

In [29]:
uni_links = {k: v for k, v in uni_links.items() if not k in keys_to_delete}

In [31]:
total_uni_links = {**uni_links, **new_uni_links}

## GET TABLES FROM THESE PAGES

In [33]:
def get_tables(href):
    r = requests.get(href)
    return pd.read_html(r.content, header=0)

In [34]:
tables = {k: get_tables(v) for k,v in total_uni_links.items()}

In [43]:
# THROW AWAY FOREIGN UNIs AND MISSING DATA
keys_to_remove = []
for k, v in tables.items():
    try:
        v[5]
    except IndexError:
        keys_to_remove.append(k)

In [44]:
tables = {k: v for k, v in tables.items() if not k in keys_to_remove}

## FIND TABLE WITH THE PROGRAMS

In [79]:
def find_rel_table(table_list):
    for table in table_list:
        if 'Képz.szint' in table.columns:
            return table

In [80]:
pot_tables = {k: find_rel_table(v) for k, v in tables.items()}

## ADD UNI ID AND CONCAT

In [105]:
for k, v in pot_tables.items():
    v['Intézmény/Kar'] = k

In [124]:
out = pd.concat(pot_tables.values(), sort=False, ignore_index=True)

## POLISH AND WRITE TO FILE

In [125]:
out.columns = [unicodedata.normalize("NFKD", col) for col in out.columns]

In [134]:
def strip_numbers_from_end(string):
    for i in range(20):
        if str(i) in string:
            string = string.rstrip(f'{str(i)})').rstrip('(')
        else:
            continue
    return string

In [135]:
out['Meghirdetett képzés - lábjegyzet nélkül'] = [strip_numbers_from_end(f) for f in out['Meghirdetett képzés']]

In [142]:
out.shape[0] - out.isnull().sum()

Képz.szint                                               4542
Munka-rend                                                4542
Fin.forma                                                 4542
Meghirdetett képzés                                     4542
Önköltség (félév)                                    4542
Képz. idő(félév)                                      4542
Kapacitásmin. < max.                                     4542
Személyes megjelenést igénylő vizsgaformák             21
Képz.terület                                            4542
Intézmény/Kar                                           4542
Érettségi vizsgakövetelmények a pontszámításhoz    4469
Pontsz. fels. okl. IGEN/NEM                               4521
Vizsgatárgyak                                              52
Meghirdetett képzés - lábjegyzet nélkül                 4542
dtype: int64

In [137]:
out.to_excel('felvi-2020-meghirdetett-kepzesek.xlsx')